In [11]:
%load_ext nb_black

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from pandas_profiling import ProfileReport

import altair as alt

from scipy import stats
from scipy.stats import norm, skew
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn


import os

os.chdir('/Users/66789/Documents/GitHub/aag-bain-f2fw/')
datafile = 'data/230428 trucking2303_shared with AAG.xlsx'

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [12]:
freq = "QUARTERLY"
df = pd.read_excel(datafile, sheet_name=freq).T
new_header = df.iloc[1] #grab the first row for the header
df = df[3:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df.drop(['Label: (SA = seasonally adjusted)'], axis=1, inplace=True)
df = df.loc[:, df.columns.notna()]
df.dropna(axis = 0, how = 'all', inplace = True)

df = df.apply(pd.to_numeric, downcast='float', errors='coerce')

if freq == "QUARTERLY":
    df['Date'] = pd.date_range('2000-01-01', '2025-12-01', freq='QS')
    df['Year'] = df['Date'].dt.year
    df['Quarter'] = df['Date'].dt.quarter
    df = df[ ['Date'] + ['Year'] + ['Quarter'] + [ col for col in df.columns if col != 'Date'] ]
else:
    df['Date'] = pd.date_range('2000-01-01', '2025-01-01', freq='MS')
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df = df[ ['Date'] + ['Year'] + ['Month'] + [ col for col in df.columns if col != 'Date'] ]


display(df.describe())



ValueError: Columns must be same length as key

<IPython.core.display.Javascript object>

In [ ]:
train_missing = (df.isnull().sum()/len(df)) *100
train_missing = train_missing.drop(train_missing[train_missing ==0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Percentage': train_missing})
missing_data

In [ ]:
# timeseries_heatmap(dataframe=df, entity_column='Site Num', sortby='Date')

profile = ProfileReport(df, tsmode=True, sortby="Date")
profile.to_file(freq+'_profile_report.html')

In [ ]:
train_spot = df[['Date', 'Total TL: Spot Rate (exc. FSC, SA)']].dropna()
train_spot.set_index('Date', inplace=True)
train_spot.index.freq='QS'
train_spot.index

train_contract = df[['Date', 'Total TL: Contract Rate (exc. FSC, SA)']].dropna()
train_contract.set_index('Date', inplace=True)
train_contract.index.freq='QS'
train_contract.index


In [ ]:
print('Contract rate')
sns.heatmap(pd.pivot_table(data=train_contract, index=train_contract.index.year, columns=train_contract.index.quarter),
           square=True,
           cmap='Blues',
           xticklabels=["Q1", "Q2", "Q3", "Q4"])



In [ ]:
print('Spot rate')
sns.heatmap(pd.pivot_table(data=train_spot, index=train_spot.index.year, columns=train_spot.index.quarter),
           square=True,
           cmap='Blues',
           xticklabels=["Q1", "Q2", "Q3", "Q4"])

In [ ]:
sns.pairplot(df)

In [ ]:
pd.plotting.lag_plot(train_spot["Total TL: Spot Rate (exc. FSC, SA)"],1)

In [ ]:
pd.plotting.lag_plot(train_spot["Total TL: Spot Rate (exc. FSC, SA)"],4)